In [130]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.signal as signal
import peakutils
import wfdb
from wfdb import processing
import ast
import neurokit2 as nk

#For signal manipulation and fast fourier transform
import scipy.signal as signal
from scipy.fftpack import fft

from sklearn.preprocessing import OneHotEncoder

## Section 3: Feature Engineering

This notebook creates the features that are used as input for the Cascading Classifier. As described in the main README file, the following types of features are created:

Statistical features: Kurtosis, used to detect signal quality, and Skew, used to detect symmetry around the R-peak.

Power and Frequency features: Fast-fourier transformed data (frequency domain), for phase and power features, and Short-time fourier transformed data (time-frequency domain), for frequency and location features.

Amplitude/Voltage features: Max, Min, and Avg peak heights, as the amplitude on the ECG paper represents the amount of voltage.

This notebook is split into several parts.

1. [Loading the data](#part-3-1-loading-the-data)
2. [Creating features](#part-3-2-creating-features)
 1. [Calculating skew and kurtosis](#part-3-2-1-calculating-skew-and-kurtosis)
 2. [Calculating short-time fourier transform](#part-3-2-2-calculating-short-time-fourier-transform)
 3. [Calculating fast fourier transform](#part-3-2-3-calculating-fast-fourier-transform)
 4. [Calculating highest, lowest, and average amplitude of the heartbeats and taking their mean](#part-3-2-4-calculating-highest-lowest-and-average-amplitude-of-the-heartbeats-and-taking-their-mean)

<a id=part-3-1-loading-the-data></a>

### Part 3.1: Loading the data

Loading the data

In [131]:
#Load raw_signals in array format
raw_signals = np.load('../data/data_analysis_files/ptb_raw_signals.npz')
raw_signals = raw_signals['arr_0']

In [132]:
raw_signals[0:1]

array([[[-0.119, -0.055,  0.064, ..., -0.026, -0.039, -0.079],
        [-0.116, -0.051,  0.065, ..., -0.031, -0.034, -0.074],
        [-0.12 , -0.044,  0.076, ..., -0.028, -0.029, -0.069],
        ...,
        [ 0.069,  0.   , -0.069, ...,  0.024, -0.041, -0.058],
        [ 0.086,  0.004, -0.081, ...,  0.242, -0.046, -0.098],
        [ 0.022, -0.031, -0.054, ...,  0.143, -0.035, -0.12 ]]],
      dtype=float16)

In [133]:
#Load raw_signals that were converted to dataframe
signals_df = pd.read_csv('../data/data_analysis_files/ptb_signals_df.csv.gz', compression='gzip')

In [134]:
signals_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.119,-0.055,0.064,0.086,-0.091,0.004,-0.069,-0.031,0.000,-0.026,-0.039,-0.079
1,-0.116,-0.051,0.065,0.083,-0.090,0.006,-0.064,-0.036,-0.003,-0.031,-0.034,-0.074
2,-0.120,-0.044,0.076,0.082,-0.098,0.016,-0.058,-0.034,-0.010,-0.028,-0.029,-0.069
3,-0.117,-0.038,0.080,0.077,-0.098,0.021,-0.050,-0.030,-0.015,-0.023,-0.022,-0.064
4,-0.103,-0.031,0.072,0.066,-0.087,0.021,-0.045,-0.027,-0.020,-0.019,-0.018,-0.058


In [161]:
meta_data_df = pd.read_csv('../data/data_analysis_files/meta_data_df.csv')

In [162]:
meta_data_df.head()

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,diagnostic_superclass
0,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,NORM
1,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr,NORM
2,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,NORM
3,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,NORM
4,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,NORM


<a id=part-3-2-creating-features></a>

### Part 3.2: Creating features

<a id=part-3-2-1-calculating-skew-and-kurtosis></a>

#### Part 3.2.1: Calculating Skew and Kurtosis

Let's calculate the skewness of one of the channels of an ecg. Apply a scipy skew function to a rolling window of the whole 10 seconds

In [135]:
#Define the scipy function and window size to be used
skew_func = sp.stats.skew
window = 1000

In [136]:
#Applying the skew function to ecg 0, whole 10 seconds, first lead, which is raw_signals[0][0:999][0]
#or signals_df[0:1000]
skew_first_ecg = pd.DataFrame.rolling(signals_df[0:1000],window).skew
print(skew_first_ecg().iloc[999])

0     2.350041
1     1.826513
2    -0.833393
3    -2.266357
4     2.084855
5     0.757849
6    -4.019669
7    -3.866994
8    -0.825758
9     2.247386
10    3.170848
11    1.842737
Name: 999, dtype: float64


Define a new function to apply functions like mean, skew, and other function names and return the results in a dataframe for that dataframe

In [137]:
def func_on_ecg(dataframe, function_name='skew'):
    """
    Input: A dataframe with 21837000 rows and 12 columns, with each row being a time step and each column being a lead
    of the ecg reading
    Output: A dataframe with 21837 rows and 12 columns, with each row being one ecg and each column being a lead
    of the ecg reading. Each value will be the applied function's values, e.g. 'mean' will retrieve the mean
    """
    output_df = pd.DataFrame(data=None)
    
    # For each of the 12-leads, do...
    for i in range(12):
    
        # Convert the dataframe into a (1000,21837) dataframe, with each column being 1 ecg for the 1000 time steps
        changed_df = pd.DataFrame(signals_df.iloc[:,i].values.reshape(21837,1000)).T
        
        #Apply the function per variable
        ecg_values = getattr(changed_df, function_name)()
        
        #Turn the above output_array into a dataframe for concatenation
        df_for_appending = pd.DataFrame(ecg_values)
        
        #Concatenate df_for_appending to the output_df
        output_df = pd.concat([output_df, df_for_appending], axis=1)
    
    #Rename columns with lead number
    output_df.columns = [0,1,2,3,4,5,6,7,8,9,10,11]
    
    return output_df
        

In [138]:
#Applying the kurtosis function
kurtosis_df = func_on_ecg(signals_df, 'kurtosis')

In [139]:
#Save kurtosis_df as a csv
kurtosis_df.to_csv('../data/features_df_folder/kurtosis_df.csv', index=False)

In [140]:
#Applying the skew function
skew_df = func_on_ecg(signals_df, 'skew')

In [141]:
#Save skew_df as a csv
skew_df.to_csv('../data/features_df_folder/skew_df.csv', index=False)

<a id=part-3-2-2-calculating-short-time-fourier-transform></a>

#### Part 3.2.2: Calculating Short-Time Fourier Transform

Let's define a function to get the short-time fourier transform for each patient in the dataframe. Sum up these values so that it works as a feature

In [153]:
def stft_on_df(dataframe):
    """
    Input: A dataframe with 21837000 rows and 12 columns, with each row being a time step and each column being a lead
    of the ecg reading
    Output: A dataframe with 21837 rows and 12 columns, with each row being one ecg and each column being a lead
    of the ecg reading. Values in this dataframe will be the sum of the short-time fourier transform, which can be
    used as a feature
    """
    output_df = pd.DataFrame(data=None)
    
    # For each of the 12-leads, do...
    for i in range(12):
    
        # Convert the dataframe into a (1000,21837) dataframe, with each column being 1 ecg for the 1000 time steps
        changed_df = pd.DataFrame(signals_df.iloc[:,i].values.reshape(21837,1000)).T
        
        #Apply the signal_timefrequency function from neurokit2 for each column. Access the 2nd value, which
        #is an array of the short-time fourier transform values
        functioned_df = changed_df.apply(lambda x : nk.signal_timefrequency(x, show = False), axis=0).iloc[2]
        
        #Initiate empty array to store the sum for the next step
        output_array = []
        
        #Sum up the short-time fourier transform values
        [output_array.append(np.sum(functioned_df[j])) for j in range(21837)]
        
        #Turn the above output_array into a dataframe for concatenation
        df_for_appending = pd.DataFrame(output_array)
        
        #Concatenate df_for_appending to the output_df
        output_df = pd.concat([output_df, df_for_appending], axis=1)
    
    #Rename columns with lead number
    output_df.columns = [0,1,2,3,4,5,6,7,8,9,10,11]
    
    return output_df
        

In [154]:
#Applying the stft_on_df function and saving the output to stft_df
stft_df = stft_on_df(signals_df)

C:\Users\junho\anaconda3\lib\site-packages\scipy\signal\spectral.py:1961: UserWarning: nperseg = 50000 is greater than input length  = 1000, using nperseg = 1000
  warnings.warn('nperseg = {0:d} is greater than input length '


In [155]:
#Saving stft_df to csv
stft_df.to_csv('../data/features_df_folder/stft_df.csv', index=False)

<a id=part-3-2-3-calculating-fast-fourier-transform></a>

#### Part 3.2.3: Calculating Fast Fourier Transform

In [156]:
def fft_on_df(dataframe):
    """
    Input: A dataframe with 21837000 rows and 12 columns, with each row being a time step and each column being a lead
    of the ecg reading
    Output: A dataframe with 21837 rows and 12 columns, with each row being one ecg and each column being a lead
    of the ecg reading. Values in this dataframe will be the sum of the fast fourier transform, which can be
    used as a feature
    """
    output_df = pd.DataFrame(data=None)
    
    # For each of the 12-leads, do...
    for i in range(12):
    
        # Convert the dataframe into a (1000,21837) dataframe, with each column being 1 ecg for the 1000 time steps
        changed_df = pd.DataFrame(signals_df.iloc[:,i].values.reshape(21837,1000)).T
        
        #Apply the signal_resample function from neurokit2 for each column. Access the 2nd value, which
        #is an array of the fast fourier transform values
        functioned_df = changed_df.apply(lambda x : nk.signal_resample(signal=x, 
                                                                       sampling_rate=100, 
                                                                       desired_sampling_rate=100,
                                                                       method='fft'), axis=0).iloc[2]
        
        #Initiate empty array to store the sum for the next step
        output_array = []
        
        #Sum up the fast fourier transform values
        [output_array.append(np.sum(functioned_df[j])) for j in range(21837)]
        
        #Turn the above output_array into a dataframe for concatenation
        df_for_appending = pd.DataFrame(output_array)
        
        #Concatenate df_for_appending to the output_df
        output_df = pd.concat([output_df, df_for_appending], axis=1)
    
    #Rename columns with lead number
    output_df.columns = [0,1,2,3,4,5,6,7,8,9,10,11]
    
    return output_df

In [157]:
#Applying the fft_on_df function
fft_df = fft_on_df(signals_df)

In [158]:
#Saving fft_df to csv
fft_df.to_csv('../data/features_df_folder/fft_df.csv', index=False)

<a id=part-3-2-4-calculating-highest-lowest-and-average-amplitude-of-the-heartbeats-and-taking-their-mean></a>

#### Part 3.2.4: Calculating highest, lowest, and average amplitude of the heartbeats and taking their mean

The amplitudes of each heartbeat represents the voltage going through the heart. Calculating the highest amplitude (normally the same as the R-peak, though not always; see channel 7 ('V2')) and lowest amplitude in a heartbeat will give a representation of the strength of the voltage. The mean, highest, and lowest amplitudes will be taken to give an overall value for the ecg.

Let's calculate the maximum amplitudes

In [144]:
def max_peak_values(dataframe):
    """
    Input: A dataframe with 21837000 rows and 12 columns, with each row being a time step and each column being a lead
    of the ecg reading. 
    Output: A dataframe with 21837 rows and 12 columns, with each row being one ecg and each column being a lead
    of the ecg reading. Values in this dataframe will be the sum of the maximum peak values.
    """
    output_df = pd.DataFrame(data=None)

    for i in range(12):
        # Convert the dataframe into a (1000,21837) dataframe, with each column being 1 ecg for the 1000 time steps
        changed_df = pd.DataFrame(dataframe.iloc[:,i].values.reshape(21837,1000)).T

        #Get the max values. Use index 0 to get the list.
        sample_max_index = changed_df.apply(lambda x: signal.find_peaks(x, distance= 90)[0])

        #For a column (a particular ecg), get the values of the sample_max_index values for that column.
        #Then, sum up the values together.
        sample_max_values = [sum(changed_df.iloc[sample_max_index[j],column_index]) for column_index, j\
                             in enumerate(range(len(sample_max_index)))]

        #Convert the array to a dataframe
        df_for_appending = pd.DataFrame(sample_max_values)

        #Concatenate df_for_appending to the output_df
        output_df = pd.concat([output_df, df_for_appending], axis=1)
    
    #Rename columns with lead number
    output_df.columns = [0,1,2,3,4,5,6,7,8,9,10,11]
    
    return (output_df)

In [145]:
#Apply max_peak_values function
max_peak_df = max_peak_values(signals_df)

In [146]:
#Save max_peak_df to csv
max_peak_df.to_csv('../data/features_df_folder/max_peak_df.csv', index=False)

Now let's invert signals_df and get the minima values

In [147]:
#Multiply signals_df by -1 to invert the signals
inverted_signals_df = signals_df * -1

In [148]:
def min_peak_values(dataframe, inverted_dataframe):
    """
    Input: A dataframe with 21837000 rows and 12 columns, with each row being a time step and each column being a lead
    of the ecg reading. An inverted dataframe with the same shape as the first dataframe and is simply the inverse of
    the original dataframe.
    Output: A dataframe with 21837 rows and 12 columns, with each row being one ecg and each column being a lead
    of the ecg reading. Values in this dataframe will be the sum of the minimum peak values.
    """
    output_df = pd.DataFrame(data=None)

    for i in range(12):
        # Convert the inverted dataframe into a (1000,21837) dataframe, 
        # with each column being 1 ecg for the 1000 time steps
        changed_inverted_df = pd.DataFrame(inverted_dataframe.iloc[:,i].values.reshape(21837,1000)).T

        # Convert the dataframe into a (1000,21837) dataframe, with each column being 1 ecg for the 1000 time steps
        changed_df = pd.DataFrame(inverted_dataframe.iloc[:,i].values.reshape(21837,1000)).T
        
        #Get the min values from inverted_dataframe. Use index 0 to get the list.
        sample_min_index = changed_inverted_df.apply(lambda x: signal.find_peaks(x, distance= 90)[0])

        #For a column (a particular ecg), get the values of the sample_min_index values for that column.
        #Then, sum up the values together.
        sample_min_values = [sum(changed_df.iloc[sample_min_index[j],column_index]) for column_index, j\
                             in enumerate(range(len(sample_min_index)))]

        #Convert the array to a dataframe
        df_for_appending = pd.DataFrame(sample_min_values)

        #Concatenate df_for_appending to the output_df
        output_df = pd.concat([output_df, df_for_appending], axis=1)
    
    #Rename columns with lead number
    output_df.columns = [0,1,2,3,4,5,6,7,8,9,10,11]
    
    return (output_df)

In [149]:
#Apply min_peak_values on inverted signals
min_peak_df = min_peak_values(signals_df, inverted_signals_df)

In [150]:
#Convert min_peak_df to csv
min_peak_df.to_csv('../data/features_df_folder/min_peak_df.csv', index=False)

In [151]:
#Calculate the average peaks
avg_peak_df = (max_peak_df - min_peak_df) /2

In [152]:
#Save avg_peak_df to csv
avg_peak_df.to_csv('../data/features_df_folder/avg_peak_df.csv', index=False)